# Random Forest Interpretation 

In [16]:
2020-7-7 14:52:43 

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

/home/eclipse/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

sns.set_style('darkgrid')

## Processing Data

In [5]:
df_raw = pd.read_feather('tmp/df_raw1')

In [40]:
df, y, nas = proc_df(df_raw, 'SalePrice', subset=100000)

In [41]:
df.shape, y.shape

((100000, 66), (100000,))

In [44]:
def split(data_set, size):
    return data_set[:size], data_set[size:]
test_size = 12000
size = len(df) - test_size

X_train, X_valid = split(df, size)
y_train, y_valid =  split(y, size)

In [43]:
X_valid.shape, X_train.shape, y_train.shape

((12000, 66), (88000, 66), (88000,))

# Feature Importance

In [45]:
rf = RandomForestRegressor(n_estimators=100, max_features=0.6, min_samples_leaf=3, oob_score=True)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=0.6, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=3,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)

In [46]:
def rmse(actuals, predicted):
    '''Returns the root mean square log error between the actual value and predicted value '''
    return np.sqrt(mean_squared_error(actuals, predicted))

def print_score(model):
    '''Prints out a series of score about the model passed 
    
    score
    =====
    The R^2 score of the training or validation set
    
    rmse
    ====
    The root mean square log error between the actual price and the predicted price
    '''
    scores = [rmse(y_train , model.predict(X_train)), rmse(y_valid, model.predict(X_valid)),
              model.score(X_train, y_train), model.score(X_valid, y_valid)
            ]
    print(f'The rmse of the train set : {scores[0]}')
    print(f'The rmse of the validation set : {scores[1]}')
    print(f'The score of the train set : {scores[2]}')
    print(f'The score of the validation set : {scores[3]}')

    # adding the oob_score
    if hasattr(model, 'oob_score_'):
        scores.append(model.oob_score_)
        print(f'The OOB_score : {scores[4]}')

In [47]:
print_score(rf)

The rmse of the train set : 0.12696816327773017
The rmse of the validation set : 0.271252560207172
The score of the train set : 0.966087488470322
The score of the validation set : 0.8650758399278538
The OOB_score : 0.8935606322993769
